# Film Analytics

## Introduction

![Movie studio with stage lights](https://smartclick.ai/wp-content/uploads/2020/08/shutterstock_1356497663-1200x529.jpg)

As our company ventures into the exciting world of cinema, understanding current trends and the industry climate is crucial. This project aims to identify specific attributes that contribute to the success of a movie, enabling us to model our business practices after these insights. By analyzing key factors and trends, we can make informed decisions and enhance our competitive edge in the film industry.

### Busines Problem

Our company has noticed that major players in the industry are producing original video content, and we want to join in on this opportunity. We have decided to launch a new movie studio, but we currently lack expertise in movie production. My task is to explore and analyze which types of films are currently performing the best at the box office. Based on these findings, I will provide actionable insights to guide the head of our new movie studio in deciding what types of films to create.

### Data

For this project, I will be using data from a combination of sources:

- [Box Office Mojo](https://www.boxofficemojo.com/)
- [IMDB](https://www.imdb.com/)
- [Rotten Tomatoes](https://www.rottentomatoes.com/)
- [TheMovieDB](https://www.themoviedb.org/)
- [The Numbers](https://www.the-numbers.com/)

## Import Libraries & Navigate Directory

In [1]:
# Import libraries
import pandas as pd
import numpy as np
import seaborn as sns
import sqlite3
import matplotlib.pyplot as plt
import zipfile
import os
import ast
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.linear_model import LinearRegression
from matplotlib import style
from sklearn.model_selection import train_test_split

In [2]:
# Navigate to the root directory
# Verify the current working directory

%cd C:/Users/emmam/Documents/Flatiron/phase_2/film-analytics/data
%pwd

C:\Users\emmam\Documents\Flatiron\phase_2\film-analytics\data


'C:\\Users\\emmam\\Documents\\Flatiron\\phase_2\\film-analytics\\data'

## Database Connection and Table Overview

In [3]:
# Connect to database and print table names

conn = sqlite3.connect('im.db')
pd.read_sql("""SELECT name FROM sqlite_master WHERE type='table';""", conn)

,name
0,movie_basics
1,directors
2,known_for
3,movie_akas
4,movie_ratings
5,persons
6,principals
7,writers


In [4]:
# Querying the database to get movie information including title, start year, average rating, number of votes, and 
# director's name then viewing results

imdb_movie_info = pd.read_sql("""SELECT primary_title, start_year, averagerating AS average_rating, numvotes AS num_votes, 
primary_name AS director
FROM persons
JOIN directors ON persons.person_id = directors.person_id
JOIN movie_basics ON directors.movie_id = movie_basics.movie_id
JOIN movie_ratings ON movie_basics.movie_id = movie_ratings.movie_id;""", conn)

imdb_movie_info

,primary_title,start_year,average_rating,num_votes,director
0,Paano na kaya,2010,6.4,77,Ruel S. Bayani
1,Paano na kaya,2010,6.4,77,Ruel S. Bayani
2,Paano na kaya,2010,6.4,77,Ruel S. Bayani
3,Paano na kaya,2010,6.4,77,Ruel S. Bayani
4,No Other Woman,2011,6.4,256,Ruel S. Bayani
...,...,...,...,...,...
181382,Late Life: The Chien-Ming Wang Story,2018,5.8,281,Frank W Chen
181383,Late Life: The Chien-Ming Wang Story,2018,5.8,281,Frank W Chen
181384,Lilli,2018,5.7,193,Prasobh Vijayan
181385,Polskie gówno,2014,5.2,215,Grzegorz Jankowski


## Load Datasets

In [5]:
# Loading datasets from CSV and TSV files into pandas DataFrames then viewing each dataframe to ensure they were loaded properly

movie_gross = pd.read_csv('bom.movie_gross.csv.gz')
movie_info = pd.read_csv('rt.movie_info.tsv.gz', delimiter='\t')
movie_reviews = pd.read_csv('rt.reviews.tsv.gz', delimiter='\t', encoding='latin-1')
movie_votes = pd.read_csv('tmdb.movies.csv.gz')
movie_budgets = pd.read_csv('tn.movie_budgets.csv.gz')

In [6]:
movie_gross.head()

,title,studio,domestic_gross,foreign_gross,year
0,Toy Story 3,BV,415000000.0,652000000,2010
1,Alice in Wonderland (2010),BV,334200000.0,691300000,2010
2,Harry Potter and the Deathly Hallows Part 1,WB,296000000.0,664300000,2010
3,Inception,WB,292600000.0,535700000,2010
4,Shrek Forever After,P/DW,238700000.0,513900000,2010


In [7]:
movie_info.head()

,id,synopsis,rating,genre,director,writer,theater_date,dvd_date,currency,box_office,runtime,studio
0,1,"This gritty, fast-paced, and innovative police...",R,Action and Adventure|Classics|Drama,William Friedkin,Ernest Tidyman,"Oct 9, 1971","Sep 25, 2001",NaN,NaN,104 minutes,NaN
1,3,"New York City, not-too-distant-future: Eric Pa...",R,Drama|Science Fiction and Fantasy,David Cronenberg,David Cronenberg|Don DeLillo,"Aug 17, 2012","Jan 1, 2013",$,"600,000",108 minutes,Entertainment One
2,5,Illeana Douglas delivers a superb performance ...,R,Drama|Musical and Performing Arts,Allison Anders,Allison Anders,"Sep 13, 1996","Apr 18, 2000",NaN,NaN,116 minutes,NaN
3,6,Michael Douglas runs afoul of a treacherous su...,R,Drama|Mystery and Suspense,Barry Levinson,Paul Attanasio|Michael Crichton,"Dec 9, 1994","Aug 27, 1997",NaN,NaN,128 minutes,NaN
4,7,NaN,NR,Drama|Romance,Rodney Bennett,Giles Cooper,NaN,NaN,NaN,NaN,200 minutes,NaN


In [8]:
movie_reviews.head()

,id,review,rating,fresh,critic,top_critic,publisher,date
0,3,A distinctly gallows take on contemporary fina...,3/5,fresh,PJ Nabarro,0,Patrick Nabarro,"November 10, 2018"
1,3,It's an allegory in search of a meaning that n...,NaN,rotten,Annalee Newitz,0,io9.com,"May 23, 2018"
2,3,... life lived in a bubble in financial dealin...,NaN,fresh,Sean Axmaker,0,Stream on Demand,"January 4, 2018"
3,3,Continuing along a line introduced in last yea...,NaN,fresh,Daniel Kasman,0,MUBI,"November 16, 2017"
4,3,... a perverse twist on neorealism...,NaN,fresh,NaN,0,Cinema Scope,"October 12, 2017"


In [9]:
movie_votes.head()

,Unnamed: 0,genre_ids,id,original_language,original_title,popularity,release_date,title,vote_average,vote_count
0,0,"[12, 14, 10751]",12444,en,Harry Potter and the Deathly Hallows: Part 1,33.533,2010-11-19,Harry Potter and the Deathly Hallows: Part 1,7.7,10788
1,1,"[14, 12, 16, 10751]",10191,en,How to Train Your Dragon,28.734,2010-03-26,How to Train Your Dragon,7.7,7610
2,2,"[12, 28, 878]",10138,en,Iron Man 2,28.515,2010-05-07,Iron Man 2,6.8,12368
3,3,"[16, 35, 10751]",862,en,Toy Story,28.005,1995-11-22,Toy Story,7.9,10174
4,4,"[28, 878, 12]",27205,en,Inception,27.920,2010-07-16,Inception,8.3,22186


In [10]:
movie_budgets.head()

,id,release_date,movie,production_budget,domestic_gross,worldwide_gross
0,1,"Dec 18, 2009",Avatar,"$425,000,000","$760,507,625","$2,776,345,279"
1,2,"May 20, 2011",Pirates of the Caribbean: On Stranger Tides,"$410,600,000","$241,063,875","$1,045,663,875"
2,3,"Jun 7, 2019",Dark Phoenix,"$350,000,000","$42,762,350","$149,762,350"
3,4,"May 1, 2015",Avengers: Age of Ultron,"$330,600,000","$459,005,868","$1,403,013,963"
4,5,"Dec 15, 2017",Star Wars Ep. VIII: The Last Jedi,"$317,000,000","$620,181,382","$1,316,721,747"


In [11]:
# Define a function to remove special characters from a specified column in a DataFrame
# Apply the function to clean the titles and primary titles in various DataFrames
# Merge multiple DataFrames on the 'title' column to create a consolidated DataFrame

def remove_special_characters(df, column_name):
    df[column_name] = df[column_name].str.replace('[^a-zA-Z0-9 ]', '', regex=True).str.lower().str.strip()
    return df

movie_gross = remove_special_characters(movie_gross, 'title')
movie_votes = remove_special_characters(movie_votes, 'title')
movie_budgets = remove_special_characters(movie_budgets, 'movie')
imdb_movie_info_grouped = remove_special_characters(imdb_movie_info, 'primary_title')

merged_data = pd.merge(movie_gross, movie_votes, how='inner', left_on='title', right_on='title')
merged_data = pd.merge(merged_data, movie_budgets, how='inner', left_on='title', right_on='movie')

df = pd.merge(imdb_movie_info, merged_data, left_on='primary_title', right_on='movie', how='inner')

df.head()

,primary_title,start_year,average_rating,num_votes,director,title,studio,domestic_gross_x,foreign_gross,year,...,popularity,release_date_x,vote_average,vote_count,id_y,release_date_y,movie,production_budget,domestic_gross_y,worldwide_gross
0,diary of a wimpy kid rodrick rules,2011,6.6,23135,David Bowers,diary of a wimpy kid rodrick rules,Fox,52700000.0,19700000,2011,...,8.961,2011-03-25,6.4,492,80,"Mar 25, 2011",diary of a wimpy kid rodrick rules,"$18,000,000","$52,698,535","$73,695,194"
1,diary of a wimpy kid rodrick rules,2011,6.6,23135,David Bowers,diary of a wimpy kid rodrick rules,Fox,52700000.0,19700000,2011,...,8.961,2011-03-25,6.4,492,80,"Mar 25, 2011",diary of a wimpy kid rodrick rules,"$18,000,000","$52,698,535","$73,695,194"
2,diary of a wimpy kid rodrick rules,2011,6.6,23135,David Bowers,diary of a wimpy kid rodrick rules,Fox,52700000.0,19700000,2011,...,8.961,2011-03-25,6.4,492,80,"Mar 25, 2011",diary of a wimpy kid rodrick rules,"$18,000,000","$52,698,535","$73,695,194"
3,diary of a wimpy kid dog days,2012,6.3,19571,David Bowers,diary of a wimpy kid dog days,Fox,49000000.0,28100000,2012,...,11.360,2012-08-03,6.2,440,17,"Aug 3, 2012",diary of a wimpy kid dog days,"$22,000,000","$49,008,662","$77,229,695"
4,diary of a wimpy kid dog days,2012,6.3,19571,David Bowers,diary of a wimpy kid dog days,Fox,49000000.0,28100000,2012,...,11.360,2012-08-03,6.2,440,17,"Aug 3, 2012",diary of a wimpy kid dog days,"$22,000,000","$49,008,662","$77,229,695"


In [12]:
# Inspecting shape to check for anamolies 

df.shape

(5751, 25)

## Define Functions

In [13]:
# Define a function to drop rows with NaNs in specified columns from a DataFrame

def drop_nans(df, column_name):
    """
    Drop NaNs from specified columns from a Pandas DataFrame.

    :param df: Pandas DataFrame from which NaNs will be dropped.
    :param column_name: Target column.
    """
    df.dropna(subset=column_name, inplace=True)

In [14]:
# Define a function to drop specified columns from a DataFrame

def drop_columns(df, columns_to_drop):
    """
    Drop specified columns from a Pandas DataFrame.

    :param df: Pandas DataFrame from which columns will be dropped.
    :param columns_to_drop: List of column names to drop.
    """
    df.drop(columns=columns_to_drop, inplace=True)

In [15]:
# Define a function to format specified columns in the DataFrame by converting currency strings to float

def format_column(df, columns):
    """
    Format specified columns in the DataFrame to convert currency strings to float.

    Args:
        df (pd.DataFrame): The DataFrame containing the columns to format.
        columns (list): A list of column names to format.

    Returns:
        pd.DataFrame: The DataFrame with specified columns formatted as floats.
    """
    for column in columns:
        if column in df.columns:
            # Convert the column to string, replace commas and dollar signs, then convert to float
            df[column] = df[column].astype(str).str.replace(',', '').str.replace('$', '').astype(float)
    return df

## Format DataFrame

In [16]:
# Using function to drop duplicates
df.drop_duplicates(subset=['title', 'release_date_x'], inplace=True)

In [17]:
# Inspecting the shape to ensure no anomolies 

df.shape

(1346, 25)

In [18]:
# Using previously defined functions to format dataframe columns
drop_nans(df, ['studio', 'domestic_gross_x'])
drop_columns(df, ['foreign_gross', 'Unnamed: 0', 'id_x', 'original_language', 'original_title', 'id_y', 
                           'release_date_y', 'domestic_gross_y'])
format_column(df, ['domestic_gross_x', 'worldwide_gross', 'production_budget'])

,primary_title,start_year,average_rating,num_votes,director,title,studio,domestic_gross_x,year,genre_ids,popularity,release_date_x,vote_average,vote_count,movie,production_budget,worldwide_gross
0,diary of a wimpy kid rodrick rules,2011,6.6,23135,David Bowers,diary of a wimpy kid rodrick rules,Fox,52700000.0,2011,"[10751, 35]",8.961,2011-03-25,6.4,492,diary of a wimpy kid rodrick rules,18000000.0,73695194.0
3,diary of a wimpy kid dog days,2012,6.3,19571,David Bowers,diary of a wimpy kid dog days,Fox,49000000.0,2012,"[35, 10751]",11.360,2012-08-03,6.2,440,diary of a wimpy kid dog days,22000000.0,77229695.0
6,diary of a wimpy kid the long haul,2017,4.4,5635,David Bowers,diary of a wimpy kid the long haul,Fox,20700000.0,2017,"[35, 10751]",10.839,2017-05-19,5.6,193,diary of a wimpy kid the long haul,22000000.0,35609577.0
8,blue valentine,2010,7.4,170089,Derek Cianfrance,blue valentine,Wein.,9700000.0,2010,"[18, 10749]",8.994,2010-12-29,6.9,1677,blue valentine,1000000.0,16566240.0
14,the place beyond the pines,2012,7.3,226765,Derek Cianfrance,the place beyond the pines,Focus,21400000.0,2013,"[18, 80]",12.960,2013-03-27,6.9,2781,the place beyond the pines,15000000.0,47011449.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5741,whiplash,2014,8.5,616916,Damien Chazelle,whiplash,SPC,13100000.0,2014,"[18, 10402]",6.298,2013-01-18,7.9,94,whiplash,3300000.0,38969037.0
5742,whiplash,2014,8.5,616916,Damien Chazelle,whiplash,SPC,13100000.0,2014,"[18, 10402]",28.784,2014-10-10,8.4,7908,whiplash,3300000.0,38969037.0
5743,la la land,2016,8.0,436070,Damien Chazelle,la la land,LG/S,151100000.0,2016,"[35, 18, 10749]",16.275,2016-12-09,7.9,10028,la la land,20000000.0,426351163.0
5745,hitman agent 47,2015,5.7,80510,Aleksander Bach,hitman agent 47,Fox,22500000.0,2015,"[28, 80, 53]",14.082,2015-08-21,5.6,1764,hitman agent 47,35000000.0,82182803.0


In [19]:
# format 'studio' column
df['studio'] = df['studio'].str.strip().str.strip('.').str.upper()

In [20]:
# Rename columns for clarity 

df.rename(columns={'domestic_gross_x': 'domestic_gross'}, inplace=True)
df.rename(columns={'release_date_x': 'release_date'}, inplace=True)

In [21]:
# Convert release date to DateTime format

df['release_date'] = pd.to_datetime(df['release_date'])

In [22]:
# Create a mapping dictionary for each movie genre

movie_genres = {
  "genres": [
    {
      "id": 28,
      "name": "action"
    },
    {
      "id": 12,
      "name": "adventure"
    },
    {
      "id": 16,
      "name": "animation"
    },
    {
      "id": 35,
      "name": "comedy"
    },
    {
      "id": 80,
      "name": "crime"
    },
    {
      "id": 99,
      "name": "documentary"
    },
    {
      "id": 18,
      "name": "drama"
    },
    {
      "id": 10751,
      "name": "family"
    },
    {
      "id": 14,
      "name": "fantasy"
    },
    {
      "id": 36,
      "name": "history"
    },
    {
      "id": 27,
      "name": "horror"
    },
    {
      "id": 10402,
      "name": "music"
    },
    {
      "id": 9648,
      "name": "mystery"
    },
    {
      "id": 10749,
      "name": "romance"
    },
    {
      "id": 878,
      "name": "science fiction"
    },
    {
      "id": 10770,
      "name": "tv movie"
    },
    {
      "id": 53,
      "name": "thriller"
    },
    {
      "id": 10752,
      "name": "war"
    },
    {
      "id": 37,
      "name": "western"
    }
  ]
}    

In [23]:
# Map genre IDs to genre names
genre_mapping = {int(genre['id']): genre['name'] for genre in movie_genres['genres']}

# Convert genre_ids to lists of integers
df['genre_ids'] = df['genre_ids'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)

# Initialize the binarizer
mlb = MultiLabelBinarizer()

# Fit and transform the genre_ids column
genre_binarized = mlb.fit_transform(df['genre_ids'])

# Correctly map columns to genre names
valid_genre_ids = [int(col) for col in mlb.classes_]
genre_names = [genre_mapping[col] for col in valid_genre_ids]

# Create a new DataFrame with the binarized genres and correct column names
genre_df = pd.DataFrame(genre_binarized, columns=genre_names)

# Add title and release_date columns to genre_df
genre_df['title_genre'] = df['title'].reset_index(drop=True)
genre_df['release_date_genre'] = df['release_date'].reset_index(drop=True)

# Align indices 
genre_df.index = df.index

# Inspect DataFrames before concatenation
print("Shape of df before concatenation:", df.shape)
print("Shape of genre_df before concatenation:", genre_df.shape)
print("First few rows of df:")
print(df.head())
print("First few rows of genre_df:")
print(genre_df.head())

# Ensure indices are aligned before concatenation
genre_df.index = df.index

# Concatenate the DataFrames
df = pd.concat([df, genre_df], axis=1)

# Inspect the resulting DataFrame
print("Shape of df after concatenation:", df.shape)
print("First few rows of df after concatenation:")
print(df.head())

Shape of df before concatenation: (1345, 17)
Shape of genre_df before concatenation: (1345, 21)
First few rows of df:
                         primary_title  start_year  average_rating  num_votes  \
0   diary of a wimpy kid rodrick rules        2011             6.6      23135   
3        diary of a wimpy kid dog days        2012             6.3      19571   
6   diary of a wimpy kid the long haul        2017             4.4       5635   
8                       blue valentine        2010             7.4     170089   
14          the place beyond the pines        2012             7.3     226765   

            director                               title studio  \
0       David Bowers  diary of a wimpy kid rodrick rules    FOX   
3       David Bowers       diary of a wimpy kid dog days    FOX   
6       David Bowers  diary of a wimpy kid the long haul    FOX   
8   Derek Cianfrance                      blue valentine   WEIN   
14  Derek Cianfrance          the place beyond the pines  FO

In [24]:
# Drop redundant columns

df.drop('title_genre', axis=1, inplace=True)
df.drop('release_date_genre', axis=1, inplace=True)

In [25]:
# Inspect the resulting DataFrame

df.head()

,primary_title,start_year,average_rating,num_votes,director,title,studio,domestic_gross,year,genre_ids,...,thriller,crime,documentary,science fiction,mystery,music,romance,family,war,tv movie
0,diary of a wimpy kid rodrick rules,2011,6.6,23135,David Bowers,diary of a wimpy kid rodrick rules,FOX,52700000.0,2011,"[10751, 35]",...,0,0,0,0,0,0,0,1,0,0
3,diary of a wimpy kid dog days,2012,6.3,19571,David Bowers,diary of a wimpy kid dog days,FOX,49000000.0,2012,"[35, 10751]",...,0,0,0,0,0,0,0,1,0,0
6,diary of a wimpy kid the long haul,2017,4.4,5635,David Bowers,diary of a wimpy kid the long haul,FOX,20700000.0,2017,"[35, 10751]",...,0,0,0,0,0,0,0,1,0,0
8,blue valentine,2010,7.4,170089,Derek Cianfrance,blue valentine,WEIN,9700000.0,2010,"[18, 10749]",...,0,0,0,0,0,0,1,0,0,0
14,the place beyond the pines,2012,7.3,226765,Derek Cianfrance,the place beyond the pines,FOCUS,21400000.0,2013,"[18, 80]",...,0,1,0,0,0,0,0,0,0,0


In [26]:
df['studio'].unique()

array(['FOX', 'WEIN', 'FOCUS', 'BV', 'RELA', 'WB', 'LGF', 'MAGN', 'PAR',
       'SONY', 'ANNAPURNA', 'SPC', 'SUM', 'TRIS', 'UNI', 'MIRA', 'ORF',
       'LG/S', 'VE', 'WB (NL)', 'BH TILT', 'IFC', 'OVER', 'FOXS', 'SABAN',
       'FD', 'CE', 'ANCH', 'IM', 'RATT', 'A24', 'FREE', 'STRAND', 'ALP',
       'SHOUT!', 'SGEM', 'MNE', 'STX', 'TRIB', 'CBS', 'NEON', 'PARV',
       'MBOX', 'P/DW', 'PFR', 'BG', 'OSCI', 'W/DIM', 'GOLD', 'ENTMP',
       '3D', 'BST', 'SMOD', 'CLEOPATRA', 'OMNI/FSR', 'XL', 'RTWC', 'PNT',
       'ELS', 'EONE', 'P4', 'JAN', 'MGM', 'DR', 'LD', 'ORCH', 'ATO',
       'FCW', 'YASH', 'DRFT', 'KE', 'EC', 'PH', 'AMAZON', 'VIV',
       'STUDIO 8', 'EROS', 'AFFIRM', 'BBC', 'LGP', 'APP', 'COHEN', 'ALC',
       'RLJ', 'RELBIG', 'GLOBAL ROAD', 'TFA', 'WHE', 'GRTINDIA', 'UTV'],
      dtype=object)

In [27]:
studio_mapping = {
    'FOXS': 'FOX',
    'WB (NL)': 'WB',
    'LG/S': 'LGF',
    'PARV': 'PAR',
    'RELBIG': 'RELA',
}

df['studio'] = df['studio'].replace(studio_mapping)
df['studio'].unique()

array(['FOX', 'WEIN', 'FOCUS', 'BV', 'RELA', 'WB', 'LGF', 'MAGN', 'PAR',
       'SONY', 'ANNAPURNA', 'SPC', 'SUM', 'TRIS', 'UNI', 'MIRA', 'ORF',
       'VE', 'BH TILT', 'IFC', 'OVER', 'SABAN', 'FD', 'CE', 'ANCH', 'IM',
       'RATT', 'A24', 'FREE', 'STRAND', 'ALP', 'SHOUT!', 'SGEM', 'MNE',
       'STX', 'TRIB', 'CBS', 'NEON', 'MBOX', 'P/DW', 'PFR', 'BG', 'OSCI',
       'W/DIM', 'GOLD', 'ENTMP', '3D', 'BST', 'SMOD', 'CLEOPATRA',
       'OMNI/FSR', 'XL', 'RTWC', 'PNT', 'ELS', 'EONE', 'P4', 'JAN', 'MGM',
       'DR', 'LD', 'ORCH', 'ATO', 'FCW', 'YASH', 'DRFT', 'KE', 'EC', 'PH',
       'AMAZON', 'VIV', 'STUDIO 8', 'EROS', 'AFFIRM', 'BBC', 'LGP', 'APP',
       'COHEN', 'ALC', 'RLJ', 'GLOBAL ROAD', 'TFA', 'WHE', 'GRTINDIA',
       'UTV'], dtype=object)

In [31]:
genre_columns = ['action', 'adventure', 'animation', 'comedy', 'crime', 'documentary', 'drama', 'family', 'fantasy', 'history', 
                'horror', 'music', 'mystery', 'romance', 'science fiction', 'thriller', 'war', 'western', 'tv movie']

genre_counts = df[genre_columns].sum().sort_values(ascending=False)
genre_counts

drama              632
comedy             459
thriller           355
action             353
adventure          254
crime              177
romance            177
science fiction    157
family             150
horror             147
fantasy            147
mystery             99
animation           98
history             58
music               40
war                 34
documentary         24
western             18
tv movie             1
dtype: int64